# Gemini PIT XML Editor
The Gemini PIT proposal tool allows you to load targets from a csv file, but requires you to input manually the observing time for those targets.  This code takes the XML file the PIT tool creates and edits it to add to the XML file the observing times that are present, but unused, in the target csv files.

In [2]:
import glob
import xml.etree.ElementTree as ET
import pandas as pd

In [6]:
filename = "Gemini_QEB.speckle"

# first, get observation times for all targets
target_times = {}
for file in glob.glob("*.csv"):
    df = pd.read_csv(file)
    target_times = {
        **target_times,
        **{name: (iq70, iq85) for _, (name, iq70, iq85) in df[["Name", "IQ70 Program Time", "IQ85 Program Time"]].iterrows()},
    }

# next, get the mapping of the nickname used in xml file to the TIC name
tree = ET.parse(f"{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[designator] = name

#finally, walk through each observation entry and add observing time to it for iq70 conditions
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = target_times[target_name]
    progTime = observation.find("progTime")
    progTime.text = str(target_time[0])
    time = observation.find("time")
    time.text = str(target_time[0])
tree.write(f"{filename}_iq70.xml")

#repeat for iq85
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = target_times[target_name]
    progTime = observation.find("progTime")
    progTime.text = str(target_time[1])
    time = observation.find("time")
    time.text = str(target_time[1])
tree.write(f"{filename}_iq85.xml")
